In [40]:
import pandas as pd
from datetime import datetime, timedelta

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [11]:
signal_df = pd.read_csv(
    "../src/main/resources/mimic2wdb/train-set/C1/signals_69435.csv",
    header=None,
    names=["time", "a", "b", "c", "d", "ABP", "f"],
)

<IPython.core.display.Javascript object>

In [22]:
signal_df["timestamp"] = signal_df["time"].apply(
    lambda x: datetime.strptime(x, "%H:%M:%S %d/%m/%Y").timestamp()
)

<IPython.core.display.Javascript object>

In [33]:
signal_df["time"] = signal_df["time"].apply(
    lambda x: datetime.strptime(x, "%H:%M:%S %d/%m/%Y")
)

<IPython.core.display.Javascript object>

In [35]:
signal_df = signal_df.set_index("time")

<IPython.core.display.Javascript object>

In [36]:
signal_df.rolling("10T")

Rolling [window=600000000000,min_periods=1,center=False,win_type=freq,axis=0]

<IPython.core.display.Javascript object>

In [108]:
def get_interpolation(before, after, current_time, df):
    if before and after:
        weight = (current_time - before) / (after - before)
        return (
            df[df["timestamp"] == before].iloc[0]["ABP"] * (1 - weight)
            + df[df["timestamp"] == after].iloc[0]["ABP"] * weight
        )
    elif before:
        return df[df["timestamp"] == before].iloc[0]["ABP"]
    elif after:
        return df[df["timestamp"] == after].iloc[0]["ABP"]
    else:
        0.0

<IPython.core.display.Javascript object>

In [109]:
list_elements = []
iterations = 0
last_window_start = None
current_datapoint = None
for item in signal_df.rolling("10T"):
    if last_window_start is None:
        last_window_start = item.iloc[0]["ABP"]
    timestamp_diff = item.iloc[-1]["timestamp"] - item.iloc[0]["timestamp"]
    current_datapoint_candidates = [
        x for x in list(item["timestamp"]) if x == last_window_start + 360.0
    ]
    if len(current_datapoint_candidates) > 0:
        current_datapoint = current_datapoint_candidates[0]
    else:
        current_datapoint = None

    if timestamp_diff == 540.0:
        if current_datapoint:
            list_elements.append(
                item[item["timestamp"] == current_datapoint][["ABP"]].iloc[0]
            )
        else:
            neighbors_before = [
                x for x in list(item["timestamp"]) if x < last_window_start + 360
            ]
            neighbors_after = [
                x for x in list(item["timestamp"]) if x > last_window_start + 360
            ]
            if len(neighbors_before) > 0:
                neighbor_before = neighbors_before[-1]
            else:
                neighbor_before = None
            if len(neighbors_after) > 0:
                neighbor_after = neighbors_after[0]
            else:
                neighbor_after = None

            interpolation = get_interpolation(
                neighbor_before, neighbor_after, last_window_start + 360, item
            )

            list_elements.append(interpolation)

    last_window_start = item.iloc[0]["timestamp"]

with open("output.txt", "w") as f:
    for item in list_elements:
        f.write(f"{item}\n")

<IPython.core.display.Javascript object>